In [1]:
# Importing packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.classification import *
import mlflow
import logging
import sys

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

stdout_handler = logging.StreamHandler(sys.stdout)
stdout_handler.setLevel(logging.DEBUG)
stdout_handler.setFormatter(formatter)

file_handler = logging.FileHandler('logs.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)


logger.addHandler(file_handler)
logger.addHandler(stdout_handler)

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina' #or 'retina'

In [4]:
# Set paths
fasta_folder = os.path.join(os.getcwd(), os.pardir, 'data', 'raw-data', 'fasta')
interim_folder = os.path.join(os.getcwd(), os.pardir, 'data', 'interim')
models_folder = os.path.join(os.getcwd(), os.pardir, 'models')
processed_folder = os.path.join(os.getcwd(), os.pardir, 'data', 'processed')
img_folder = os.path.join(os.getcwd(), os.pardir, 'reports', 'figures')

In [5]:
# Define organisms dataset to be used
dataset_names = (
        'Arabidopsis_non_tata',
        'Arabidopsis_tata',
        'Bacillus',
        'Ecoli',
        'Human_non_tata',
        'Mouse_non_tata',
        'Mouse_tata',
    )

In [6]:
# Utility functions

def get_organism_props_dataframes(organism_path: str, extension: str = 'feather'):
    # Get all properties files from organism folder
    props_files = [f'{organism_path}/{file}' for file in os.listdir(organism_path) if file.startswith('df_data_prop_')]

    # Get dataframes for each property
    props = list()
    for prop_idx in range(len(props_files)):
        prop_file = f'{organism_path}/df_data_prop_{prop_idx}.{extension}'  # Get property file path
        if os.path.exists(prop_file): # Check if property file exists
            if extension == 'feather': # Feather is faster than csv but not compatible with other libraries
                df_data = pd.read_feather(prop_file)
            elif extension == 'csv': # CSV is more compatible with other libraries
                df_data = pd.read_csv(prop_file, index_col=False)
            else:
                raise ValueError('Extension not supported.')
            props.append(df_data) # Append dataframe to list
        else:
            print(f'Property {prop_idx} {extension} file not found.')

    return props

def get_bp_positions(seq_len: int, step: int = 10):
    # Set range of bp positions (upstream, TSS, downstream)
    if seq_len == 80:
        _range = np.arange(-60, 20, step)
    elif seq_len == 79:
        _range = np.arange(-60, 19, step)
    elif seq_len == 250:
        _range = np.arange(-200, 50, step)
    elif seq_len == 249:
        _range = np.arange(-200, 49, step)
    else:
        raise ValueError('Sequence length not supported.')
    return _range

def get_props_names(kmer_type: str = 'dinuc'):
    kmers_values_folder = os.path.join(os.getcwd(), os.pardir, 'data', 'raw-data',
                                       'physicochemical-properties-reference',
                                           f'original-{kmer_type}.tsv')
    kmer_df = pd.read_csv(kmers_values_folder, sep='\t', index_col=0)
    props_names = kmer_df.index.tolist()
    return props_names

In [7]:
# Set kmer type, get properties names and number of properties for that kmer type
kmer_types = ('dinuc', 'trinuc')                                                                                        # Define kmer types
props_names = {'dinuc': get_props_names(kmer_type=kmer_types[0]), 'trinuc': get_props_names(kmer_type=kmer_types[1])}   # Get properties names
props_num = {'dinuc': len(props_names['dinuc']), 'trinuc': len(props_names['trinuc'])}                                  # Get number of properties

In [8]:
print(f"Dinucleotide properties ({props_num['dinuc']}):\n {props_names['dinuc']}", end='\n\n')
print(f"Trinucleotide properties ({props_num['trinuc']}):\n {props_names['trinuc']}")

Dinucleotide properties (38):
 ['Base stacking', 'Protein induced deformability', 'B-DNA twist', 'Dinucleotide GC Content', 'A-philicity', 'Propeller twist', 'Duplex stability(free energy)', 'Duplex tability(disrupt energy)', 'DNA denaturation', 'Bending stiffness', 'Protein DNA twist', 'Stabilising energy of Z-DNA', 'Aida_BA_transition', 'Breslauer_dG', 'Breslauer_dH', 'Breslauer_dS', 'Electron_interaction', 'Hartman_trans_free_energy', 'Helix-Coil_transition', 'Ivanov_BA_transition', 'Lisser_BZ_transition', 'Polar_interaction', 'SantaLucia_dG', 'SantaLucia_dH', 'SantaLucia_dS', 'Sarai_flexibility', 'Stability', 'Stacking_energy', 'Sugimoto_dG', 'Sugimoto_dH', 'Sugimoto_dS', 'Watson-Crick_interaction', 'Twist', 'Tilt', 'Roll', 'Shift', 'Slide', 'Rise']

Trinucleotide properties (12):
 ['Bendability (DNAse) ', 'Bendability (consensus)', 'Trinucleotide GC Content', 'Nucleosome positioning', 'Consensus_roll', 'Consensus_Rigid', 'Dnase I', 'Dnase I-Rigid', 'MW-Daltons', 'MW-kg', 'Nucleoso

Get dataframes for each organism and each kmer type ('dinuc' and 'trinuc') and store them in a dictionary of dictionaries
 of dataframes (features)

In [9]:
# Get dataframes for each organism and each kmer type (dinuc and trinuc) and store them in a dictionary of
# dictionaries of dataframes (features)
features = {x: {'dinuc': None, 'trinuc': None} for x in dataset_names}                          # Initialize dictionary
for dataset_name in dataset_names:                                                              # Iterate over organisms
    for kmer_type in kmer_types:                                                                # Iterate over kmer types
        feature_folder = os.path.join(interim_folder, kmer_type, f'{dataset_name}-original')    # Get folder path
        features[dataset_name][kmer_type] = get_organism_props_dataframes(feature_folder)       # Get dataframes
        print(f'{dataset_name} - {kmer_type} - {len(features[dataset_name][kmer_type])}')       # Print number of properties

Arabidopsis_non_tata - dinuc - 38
Arabidopsis_non_tata - trinuc - 12
Arabidopsis_tata - dinuc - 38
Arabidopsis_tata - trinuc - 12
Bacillus - dinuc - 38
Bacillus - trinuc - 12
Ecoli - dinuc - 38
Ecoli - trinuc - 12
Human_non_tata - dinuc - 38
Human_non_tata - trinuc - 12
Mouse_non_tata - dinuc - 38
Mouse_non_tata - trinuc - 12
Mouse_tata - dinuc - 38
Mouse_tata - trinuc - 12


# Experiment to collect data for all properties and all organisms using PyCaret models
PyCaret and MLFlow tracking and logging experiments to MLFlow server (http://localhost:5000) and
artifacts to local folder (mlruns)

In [ ]:
iter_idx = 0
results_df = pd.DataFrame(columns=['Organism', 'ModelName', 'ModelID', 'K-mer', 'PropertyIdx', 'PropertyName',
                                   'Accuracy_avg', 'AUC_avg', 'F1_avg', 'Precision_avg', 'Recall_avg', 'Kappa_avg',
                                   'MCC_avg', 'MCC_max', 'MCC_min', 'TT (s)'])

results_path = os.path.join(processed_folder, 'exp-results.csv')
results_df.to_csv(results_path, index=False, mode='w')

for dataset_idx, dataset_name in enumerate(dataset_names):
    log_msg = f'> Organism {dataset_idx} - {dataset_name}'
    logger.info(log_msg)

    for k_mer_idx, k_mer_type in enumerate(kmer_types):
        # print(f'K-mer {k_mer_type}')
        for prop_idx, prop_name in enumerate(props_names[k_mer_type]):
            # print(f'Property {prop_idx} - {prop_name}')

            # Get dataframe with features for a given property and organism (dataset) and kmer type (dinuc or trinuc)
            input_features = features[dataset_name][k_mer_type][prop_idx]

            # Set up the experiment
#             exp_name = f'{dataset_name}-{k_mer_type}-{prop_idx}'
            exp_name = f'{dataset_name}'

            logger.info(f'>> ({iter_idx}) Starting experiment: {exp_name}...')

            iter_idx += 1 # increment experiment index

            # Initialize the experiment with PyCaret and MLFlow tracking and logging experiments to MLFlow server
            # (http://localhost:5000) and artifacts to local folder (mlruns)
            exp = setup(data=input_features, target='y', session_id=123, verbose=False, log_experiment=True,
                        experiment_name=exp_name, html=False, log_plots=True, fold_strategy='stratifiedkfold',
                        fold=5, preprocess=False) # log_profile=True, profile=True, profile_kwargs={'silent': True}

            # Set experiment tags for MLFlow
            tags = {'Dataset': {dataset_name}, 'kmer': k_mer_type, 'prop_idx': prop_idx, 'prop_name': prop_name}
            mlflow.set_tags(tags) # set experiment tags for MLFlow
            mlflow.set_tag("mlflow.runName", f"{k_mer_type}-{prop_idx}") # set run name

            models_types_df = models() # get models dataframe (PyCaret) with model types and names

            # compare models and select the best (PyCaret)
            best_model = compare_models(
                sort='MCC',
                experiment_custom_tags={'prop': f'{k_mer_type}_{prop_idx}'}
            )

            metrics_df = pull() # get metrics dataframe
            model_idx = metrics_df.iloc[0,:].name # get index of best model
            model_name = metrics_df.iloc[0,:]['Model'] # get name of best model
            model_mcc_avg = metrics_df.iloc[0,:]['MCC'] # get average MCC of best model
            model_mcc_max = metrics_df.loc[:,'MCC'].max() # get max MCC of best model
            model_mcc_min = metrics_df.loc[:,'MCC'].min() # get min MCC of best model

            logger.info(f'Best model: {model_name} - {model_idx} - MCC AVG: {model_mcc_avg}')
            # print(metrics_df) # print metrics for all models


            # tune model (PyCaret) - optimize MCC metric (default) - 5-fold CV (default)
            tuned_model = tune_model(
                best_model,
                optimize='MCC',
                fold=5,
                n_iter=10,
                search_library='optuna',
                search_algorithm='tpe',
                verbose=False,
                tuner_verbose=False,
            )

            tunned_metrics_df = pull() # get metrics dataframe for tunned model
            tunned_mcc_avg = tunned_metrics_df.iloc[-2,:]['MCC'] # get MCC of best tunned model
            tunned_mcc_max = tunned_metrics_df.iloc[:-2,:]['MCC'].max() # get max MCC of best model
            tunned_mcc_min = tunned_metrics_df.iloc[:-2,:]['MCC'].min() # get min MCC of best model

            print(f'Tunned model Average MCC: {tunned_mcc_avg}')
            # print(tunned_metrics_df) # print metrics for tunned model


            tags = {'Model Name': model_name, 'Model ID': model_idx, 'Prop Name': prop_name}
            mlflow.set_tags(tags) # set model tags for MLFlow

            mlflow.log_metric('MCC-avg', tunned_mcc_avg) # log best tunned avg MCC
            mlflow.log_metric('MCC-max', tunned_mcc_max) # log best tunned max MCC
            mlflow.log_metric('MCC-min', tunned_mcc_min) # log best tunned min MCC


            # Save best tuned model
            if not os.path.exists(models_folder):
                os.makedirs(models_folder)
            tunned_model_path = f'{exp_name}-{k_mer_type}-{prop_idx}-{model_idx}-tunned-{str(tunned_mcc_avg)}'
            tunned_model_path = os.path.join(models_folder, tunned_model_path)
            save_model(tuned_model, tunned_model_path) # save best tuned model (PyCaret)
            mlflow.log_artifact(f'{tunned_model_path}.pkl') # log experiment results


            row_df = pd.DataFrame({'Organism': dataset_name, 'kmer': k_mer_type, 'Property': prop_name,
                                   'Model': model_name, 'Model ID': model_idx, 'MCC_avg': model_mcc_avg,
                                   'MCC_max': model_mcc_max, 'MCC_min': model_mcc_min, 'TT (s)': 0}, index=[0])
            row_df.to_csv(results_path, mode='a', header=False, index=False) # save results to CSV file

            logger.info(f'>> Ending experiment: {exp_name}...')

            # break

    log_msg = f'> Ending organism {dataset_idx} - {dataset_name}...'
    logger.info(log_msg)

2023-05-19 01:18:05,025 | INFO | > Organism 0 - Arabidopsis_non_tata
2023-05-19 01:18:05,025 | INFO | >> (0) Starting experiment: Arabidopsis_non_tata...


2023/05/19 01:18:05 INFO mlflow.tracking.fluent: Experiment with name 'Arabidopsis_non_tata' does not exist. Creating a new experiment.


                                    Model  Accuracy     AUC  Recall   Prec.  \
lightgbm  Light Gradient Boosting Machine    0.8888  0.9479  0.7830  0.8770   
gbc          Gradient Boosting Classifier    0.8611  0.9258  0.7058  0.8608   
ada                  Ada Boost Classifier    0.8319  0.8953  0.7116  0.7759   
lr                    Logistic Regression    0.8099  0.8653  0.6526  0.7554   
lda          Linear Discriminant Analysis    0.8103  0.8654  0.6475  0.7592   
ridge                    Ridge Classifier    0.8105  0.0000  0.6371  0.7664   
qda       Quadratic Discriminant Analysis    0.8100  0.8816  0.5805  0.8064   
nb                            Naive Bayes    0.7919  0.8456  0.6886  0.6961   
rf               Random Forest Classifier    0.7847  0.8983  0.3987  0.9268   
svm                   SVM - Linear Kernel    0.7777  0.0000  0.5245  0.7973   
dt               Decision Tree Classifier    0.7289  0.7030  0.6223  0.5976   
et                 Extra Trees Classifier    0.7346 

[LightGBM] [Warning] bagging_fraction is set=0.7057186786939675, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7057186786939675
[LightGBM] [Warning] feature_fraction is set=0.630612536222984, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.630612536222984
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.9119780621598002, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9119780621598002
[LightGBM] [Warning] feature_fraction is set=0.515266122136092, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.515266122136092
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.40291526724806226, subsample=1.0 will be ignored. Current value: bagging_fraction=0.40291526724806226
[LightGBM] [Warning] feature_fraction

[LightGBM] [Warning] bagging_fraction is set=0.7057186786939675, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7057186786939675
[LightGBM] [Warning] feature_fraction is set=0.630612536222984, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.630612536222984
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.9409807381013622, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9409807381013622
[LightGBM] [Warning] feature_fraction is set=0.9063949685646929, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9063949685646929
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.9567811430072662, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9567811430072662
[LightGBM] [Warning] feature_fraction

...

[LightGBM] [Warning] bagging_fraction is set=0.6091944497792138, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6091944497792138
[LightGBM] [Warning] feature_fraction is set=0.7674244674573488, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7674244674573488
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.6223530924465498, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6223530924465498
[LightGBM] [Warning] feature_fraction is set=0.8971107648821304, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8971107648821304
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.6223530924465498, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6223530924465498
[LightGBM] [Warning] feature_fracti

[LightGBM] [Warning] bagging_fraction is set=0.676476537573909, subsample=1.0 will be ignored. Current value: bagging_fraction=0.676476537573909
[LightGBM] [Warning] feature_fraction is set=0.46338068191690096, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.46338068191690096
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.6885710181118689, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6885710181118689
[LightGBM] [Warning] feature_fraction is set=0.8583939730024278, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8583939730024278
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.5554791717553834, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5554791717553834
[LightGBM] [Warning] feature_fracti

                                    Model  Accuracy     AUC  Recall   Prec.  \
lightgbm  Light Gradient Boosting Machine    0.8880  0.9501  0.7733  0.8829   
gbc          Gradient Boosting Classifier    0.8679  0.9336  0.7210  0.8682   
ada                  Ada Boost Classifier    0.8327  0.8962  0.7162  0.7751   
rf               Random Forest Classifier    0.7960  0.9053  0.4411  0.9152   
nb                            Naive Bayes    0.7753  0.8372  0.6862  0.6642   
lda          Linear Discriminant Analysis    0.7811  0.8263  0.5843  0.7196   
ridge                    Ridge Classifier    0.7814  0.0000  0.5727  0.7266   
lr                    Logistic Regression    0.7799  0.8265  0.5850  0.7160   
svm                   SVM - Linear Kernel    0.7456  0.0000  0.5877  0.6835   
et                 Extra Trees Classifier    0.7505  0.8916  0.2829  0.9449   
dt               Decision Tree Classifier    0.7412  0.7139  0.6284  0.6174   
qda       Quadratic Discriminant Analysis    0.7555 

[LightGBM] [Warning] bagging_fraction is set=0.5834875433351111, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5834875433351111
[LightGBM] [Warning] feature_fraction is set=0.6468668653179099, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6468668653179099
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.5833310089718523, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5833310089718523
[LightGBM] [Warning] feature_fraction is set=0.9441956305739366, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9441956305739366
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.8265975093218108, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8265975093218108
[LightGBM] [Warning] feature_fracti

[LightGBM] [Warning] bagging_fraction is set=0.8357894935706773, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8357894935706773
[LightGBM] [Warning] feature_fraction is set=0.425437378618089, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.425437378618089
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=0.4805343780719675, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4805343780719675
[LightGBM] [Warning] feature_fraction is set=0.6141120408831521, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6141120408831521
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=0.6597482418844614, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6597482418844614
[LightGBM] [Warning] feature_fraction

# PyCaret experiment - Base case

In [ ]:
# Define property, dataset and kmer type to be used
prop_idx = 0
dataset_name = 'Bacillus'
kmer_type = 'dinuc'
print(f'Property {prop_idx} - {props_names["dinuc"][prop_idx]}')
# Select property dataframe for a given organism and kmer type
data = features[dataset_name][kmer_type][prop_idx]

new_cols = [f'{x}' for x in get_bp_positions(data.shape[1]-1, step=1)] + ['y'] # Get new columns names
data.columns = new_cols # Set new columns names

data

In [ ]:
# Set up the experiment
exp_name = f'{dataset_name}-{kmer_type}-{prop_idx}'
exp = setup(data=data, target='y', session_id=123, verbose=True, log_experiment=True, experiment_name=exp_name)

In [ ]:
# View all models available in the library (pycaret.classification)
models()

In [ ]:
# Compare models and select the best 3 models based on MCC
best_models = compare_models(sort='MCC', n_select=5)

In [ ]:
lightgbm = best_models[1]
lightgbm
lightgbm_tuned = tune_model(lightgbm, optimize='MCC')
pull()

In [ ]:
metrics_df = pull()
metrics_df
# predict_model(lightgbm_tuned)
# pull()

In [ ]:
metrics_df.iloc[:-2, -1].min()

# Interpretation of the model

In [ ]:
plot_model(lightgbm_tuned, 'dimension')
interpret_model(lightgbm_tuned)

In [ ]:
# !mlflow ui

In [ ]:
x = np.arange(5)
y_mean = np.array([3, 5, 4, 7, 6])
y_min = np.array([2, 4, 3, 6, 5])
y_max = np.array([4, 6, 5, 8, 7])
y_error = np.array([y_mean - y_min, y_max - y_mean])

plt.errorbar(x, y_mean, yerr=y_error, fmt='o', color='black', capsize=0, capthick=0,
             marker='o', markersize=0, mec='cornflowerblue', mfc='lightblue', ecolor='cornflowerblue',
             elinewidth=1)

# Customize the markers for the minimum, maximum, and mean values
plt.plot(x, y_min, 'v', color='royalblue', markersize=8, label='Min')
plt.plot(x, y_max, '^', color='royalblue', markersize=8, label='Max')
plt.plot(x, y_mean, 'o', color='royalblue', markersize=6, label='Mean')

plt.grid(False)

plt.legend()
plt.show()

In [ ]:
# Example data
score_name = 'MCC'
datasets = ['D1', 'D2', 'D3', 'D4']
algorithms = ['SVM', 'Random Forest']
algorithms_colors = ['cornflowerblue', 'sandybrown']
means = np.array([[0.85, 0.89], [0.75, 0.82], [0.93, 0.95], [0.91, 0.94]])
mins = np.array([[0.80, 0.85], [0.65, 0.75], [0.90, 0.92], [0.88, 0.90]])
maxs = np.array([[0.90, 0.92], [0.80, 0.88], [0.96, 0.98], [0.94, 0.96]])

# Set up plot
fig, ax = plt.subplots()
width = 0.35
ind = np.arange(len(datasets))

# Plot means
# for alg_idx, alg_name in enumerate(algorithms):
#     means_alg = means[:, alg_idx]
#     ax.bar(ind + alg_idx*width, means_alg, width, label=alg_name)

# Add errorbars
for alg_idx, alg_name in enumerate(algorithms):
    mins_alg = mins[:, alg_idx]
    maxs_alg = maxs[:, alg_idx]
    _color = algorithms_colors[alg_idx]
    ax.errorbar(ind + alg_idx*width, means[:, alg_idx], yerr=[means[:, alg_idx] - mins_alg, maxs_alg - means[:, alg_idx]],
                fmt='o', capsize=3, capthick=1, ecolor=_color, label=alg_name, elinewidth=1,
                markeredgewidth=1, mec=_color, mfc='white', markersize=6, marker='o')

# Add labels, title, legend, and grid
ax.set_ylabel(f'{score_name}')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(datasets)
ax.set_title(f'{score_name} by Dataset and Algorithm')
ax.legend(algorithms, loc='best', fancybox=True, shadow=True)
ax.grid(False)

plt.show()